In [ ]:
import spacy
import torch
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split

data = pd.read_csv('/content/data.csv')
data.info()
data

In [ ]:
!pip install transformers
!pip install sckit-learn

ERROR: Could not find a version that satisfies the requirement sckit-learn (from versions: none)
ERROR: No matching distribution found for sckit-learn


In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
# Split the data into training and testing sets
X = data["Ulasan IMDB"]
Y = data["Sifat Ulasan"]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=15)

# Load pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

# Define a function to extract semantic features
def extract_semantic_features(text):
    # Tokenize the input text
    tokens = tokenizer(text, padding=True, truncation=True, return_tensors="pt")

    # Get the BERT embeddings for the text
    with torch.no_grad():
        outputs = model(**tokens)

    # Extract features from BERT's hidden states
    features = outputs.last_hidden_state.mean(dim=1)  # Mean pooling over tokens

    return features

# Extract semantic features for the training set
X_train_semantic = [extract_semantic_features(text) for text in X_train]
X_train_semantic = torch.cat(X_train_semantic).numpy()

# Extract semantic features for the testing set
X_test_semantic = [extract_semantic_features(text) for text in X_test]
X_test_semantic = torch.cat(X_test_semantic).numpy()


In [ ]:
X = data["Ulasan IMDB"]
Y = data["Sifat Ulasan"]

X_train_syntactic, X_test_syntactic, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=15)
X_train_semantic, X_test_semantic, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=9)

# Fungsi ekstraksi fitur sintaktik
def extract_syntactic_features(text):
    doc = nlp(text)
    noun_count = len([token for token in doc if token.pos_ == "NOUN"])
    return noun_count

# Ekstraksi fitur sintaktik untuk data pelatihan
X_train_syntactic = [extract_syntactic_features(text) for text in X_train_syntactic]
X_train_syntactic = np.array(X_train_syntactic).reshape(-1, 1)

# Ekstraksi fitur sintaktik untuk data uji
X_test_syntactic = [extract_syntactic_features(text) for text in X_test_syntactic]
X_test_syntactic = np.array(X_test_syntactic).reshape(-1, 1)

# Ekstraksi fitur semantik untuk data pelatihan
X_train_semantic = [extract_semantic_features(text) for text in X_train_semantic]
X_train_semantic = np.array(X_train_semantic).reshape(-1, 1)

# Ekstraksi fitur semantik untuk data uji
X_test_semantic = [extract_semantic_features(text) for text in X_test_semantic]
X_test_semantic = np.array(X_test_semantic).reshape(-1, 1)


NameError: ignored

In [ ]:
# Membuat dan melatih model K-NN
k = 3  # Jumlah tetangga terdekat
KNN = KNeighborsClassifier(n_neighbors=k)
KNN.fit(X_train_syntactic, Y_train)

# Membuat dan melatih model SVM
SVM = SVC(kernel='linear')
SVM.fit(X_train_syntactic, Y_train)

# Membuat dan melatih model Naive Bayes
NB = MultinomialNB()
NB.fit(X_train_syntactic, Y_train)

# Membuat dan melatih model Random Forest
RF = RandomForestClassifier()
RF.fit(X_train_syntactic, Y_train)

# Prediksi kategori untuk data uji
y_pred_SVM = SVM.predict(X_test_syntactic)
y_pred_KNN = KNN.predict(X_test_syntactic)
Y_pred_NB = NB.predict(X_test_syntactic)
Y_pred_RF = RF.predict(X_test_syntactic)

# Menghitung akurasi
accuracy_SVM = accuracy_score(Y_test, y_pred_SVM)
print("Akurasi SVM: {:.2f}%".format(accuracy_SVM * 100))

accuracy_KNN = accuracy_score(Y_test, y_pred_KNN)
print("Akurasi K-NN: {:.2f}%".format(accuracy_KNN * 100))

accuracy_NB = accuracy_score(Y_test, Y_pred_NB)
print("Akurasi Naive Bayes: {:.2f}%".format(accuracy_NB * 100))

accuracy_RF = accuracy_score(Y_test, Y_pred_RF)
print("Akurasi Random Forest: {:.2f}%".format(accuracy_RF * 100))

Akurasi SVM: 50.00%
Akurasi K-NN: 50.00%
Akurasi Naive Bayes: 50.00%
Akurasi Random Forest: 50.00%


In [ ]:
# Ekstraksi fitur semantic dengan TF-IDF
tfidf_vectorizer = TfidfVectorizer()
X_train_semantic = tfidf_vectorizer.fit_transform(Y_train)
X_test_semantic = tfidf_vectorizer.transform(Y_test)



In [ ]:
# Membuat dan melatih model K-NN
k = 4  # Jumlah tetangga terdekat
KNN = KNeighborsClassifier(n_neighbors=k)
KNN.fit(X_train_semantic, Y_train)

# Membuat dan melatih model SVM
SVM = SVC(kernel='linear')
SVM.fit(X_train_semantic, Y_train)

# Membuat dan melatih model Naive Bayes
NB = MultinomialNB()
NB.fit(X_train_semantic, Y_train)

# Membuat dan melatih model Random Forest
RF = RandomForestClassifier()
RF.fit(X_train_semantic, Y_train)


# Prediksi kategori untuk data uji
y_pred_SVM = SVM.predict(X_test_semantic)
y_pred_KNN = KNN.predict(X_test_semantic)
Y_pred_NB = NB.predict(X_test_semantic)
Y_pred_RF = RF.predict(X_test_semantic)

# Menghitung akurasi
accuracy_SVM = accuracy_score(Y_test, y_pred_SVM)
print("Akurasi SVM: {:.2f}%".format(accuracy_SVM * 100))

accuracy_KNN = accuracy_score(Y_test, y_pred_KNN)
print("Akurasi K-NN: {:.2f}%".format(accuracy_KNN * 100))

accuracy_NB = accuracy_score(Y_test, Y_pred_NB)
print("Akurasi Naive Bayes: {:.2f}%".format(accuracy_NB * 100))

accuracy_RF = accuracy_score(Y_test, Y_pred_RF)
print("Akurasi Random Forest: {:.2f}%".format(accuracy_RF * 100))

ValueError: ignored